# Select the Customer to Predict Next Purchase

In [ ]:
customer_id = 15996 #15362 #13085 #15996

In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from IPythonDisplayFireworks import Fireworks
from wordcloud import WordCloud
import warnings
from IPython.display import display, HTML
warnings.filterwarnings('ignore')

# Load the model from the pickle file
with open('propensity_model_monthly.pkl', 'rb') as file:
    monthly_model = pickle.load(file)
with open('propensity_model_90days.pkl', 'rb') as file:
    days90_model = pickle.load(file)
# Read the csv
customer_class1=pd.read_csv('customer_class1.csv')
X1 = customer_class1.drop(columns = ['NextPurchaseDayRange','NextPurchaseDay','CustomerID'])
customer_bhvr_dt=pd.read_parquet('customer_bhvr_dt.parquet')

In [ ]:
customer_id=int(customer_id)


In [ ]:
def days_proba(customer_id,logit_model, monthly_model, customer_class1,X):
    # Create a bar plot of the data
    
    if  logit_model.predict_proba(customer_class1.loc[customer_class1['CustomerID']==customer_id,X.columns])[0][0]<0.5:
        
        
        display(HTML(f'<h1 style="text-align:center; color:Red;">Hurray! We can expect this customer soon!</h1>'))
        data = monthly_model.predict_proba(customer_class1.loc[customer_class1['CustomerID']==customer_id,X.columns])[0][:-1]
        fig = px.bar(x=['0-30 Days','30-60 Days','60-90 Days'], y=data, labels={'x':'Days', 'y':'Probability'})
        fig.update_layout(title='Probability distribution of buying in next 3 months', plot_bgcolor='white')
        fig.show()
        Fireworks()
    else:
        display(HTML(f'<h1 style="text-align:center; color:Red;">Oops! Looks like we don\'t expect this customer in next 90 days</h1></br></br></br>'))
        
days_proba(customer_id, days90_model, monthly_model, customer_class1,X1)


In [ ]:
display(HTML(f'<h1 style="text-align:center; color:Red;">Let\'s analyze the past data of this customer.</h1></br>'))

In [ ]:
# Calculate revenue for each month
customer_bhvr_dt['InvoiceDate']=pd.to_datetime(customer_bhvr_dt['InvoiceDate'])
temp = customer_bhvr_dt[customer_bhvr_dt['CustomerID']==customer_id]

temp['InvoiceYearMonth'] = temp['InvoiceDate'].map(lambda date: 100*date.year + date.month)
temp['Revenue'] = temp['UnitPrice'] * temp['Quantity']

# Calculate monthly revenue
temp_revenue = temp.groupby('InvoiceYearMonth')['Revenue'].sum().reset_index()

# Display revenue statistics
#print(pd.DataFrame(temp_revenue['Revenue'].describe()))

# Create a line plot for monthly revenue
line_fig = px.line(temp_revenue, 
                   x="InvoiceYearMonth", 
                   y="Revenue", 
                   title="Monthly Revenue for Customer: {} from Nov. 2009 to Aug. 2011".format(customer_id), 
                   template="plotly_white",
                   labels={"InvoiceYearMonth": "Invoice Year-Month", "Revenue": "Monthly Revenue"}
                  )

line_fig.update_layout(title_x=0.5, 
                       showlegend=False, 
                       xaxis={"type": "category"}
                      )


In [ ]:

# Get the description counts
description_counts = temp['Description'].value_counts()

# Create a dictionary of description counts
description_dict = dict(description_counts)

# Create a word cloud from the description counts
wordcloud = WordCloud(width=800, height=400, background_color='white')
wordcloud.generate_from_frequencies(description_dict)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title(f"Historical Purchases for Customer: {customer_id}")
plt.axis('off')
plt.show()


In [ ]:

def product_reco(df_data, selected_user):
    import pandas as pd
    from sklearn.metrics.pairwise import cosine_similarity
    # Pivot the dataset to create a matrix of transactions
    matrix = df_data.pivot_table(index='CustomerID', columns='ProductCode', values='Quantity', fill_value=0)
    matrix.index=matrix.index.astype('int')
    
    # Get the top 10 similar users
    cosine_sim = cosine_similarity(matrix)
    cosine_similar_users = dict(zip(matrix.index, cosine_sim))
    similar_users = list(zip(matrix.index, cosine_similar_users[selected_user]))
    sorted_similar_users = sorted(similar_users,key=lambda x:x[1],reverse=True)[1:11]
    
    # Get the products bought by the similar users
    products = []
    for user in sorted_similar_users:
        products += list(df_data[df_data['CustomerID']==user[0]]['Description'])

    # Get the top 10 recommended products
    recommended_products = pd.Series(products).value_counts().head(10).index.tolist()
    
    #print("Similar User to Customer: ",selected_user)
    #print([user[0] for user in sorted_similar_users])
    
    #print("Recommended Products for Customer: ",selected_user)
    
    #print(recommended_products)
    return recommended_products,[user[0] for user in sorted_similar_users]

recommendations,similar_users=product_reco(customer_bhvr_dt, customer_id)


In [ ]:
# Create an empty string to store the HTML code
recommend = ''
dots=''
sim_users=''
counter=1
# Loop over the recommendations list
for recommendation in recommendations:
    # Append a div element for each recommendation to the HTML string
    recommend += f'<div class="mySlides"><q>{recommendation}</q></div>'
    dots+= f'<span class="dot" onclick="currentSlide(' +str(counter)+')"></span>'
    counter=counter+1
    
for user in similar_users:
    sim_users+=f'<span class="tag-cloud">{user}</span>'

In [ ]:
html ="""
<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
.tag-cloud {
  display: inline-block;
  color: white;
  padding: 8px 20px;
  font-family: Arial;
  border-radius: 25px;
  background-color: #2196F3;
  margin-top: 8px;
}
</style>
</head>
<body>

<h1>Similar Customers</h1>
"""+ sim_users + """

</body>
</html>
"""
# Display the HTML code in a Jupyter cell
display(HTML(html))

In [ ]:
html ="""
<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
* {box-sizing: border-box}
body {font-family: Verdana, sans-serif; margin:0}

/* Slideshow container */
.slideshow-container {
  position: relative;
  background: #A8FC9B;
}

/* Slides */
.mySlides {
  display: none;
  padding: 80px;
  text-align: center;
}

/* Next & previous buttons */
.prev, .next {
  cursor: pointer;
  position: absolute;
  top: 50%;
  width: auto;
  margin-top: -30px;
  padding: 16px;
  color: #888;
  font-weight: bold;
  font-size: 20px;
  border-radius: 0 3px 3px 0;
  user-select: none;
}

/* Position the "next button" to the right */
.next {
  position: absolute;
  right: 0;
  border-radius: 3px 0 0 3px;
}

/* On hover, add a black background color with a little bit see-through */
.prev:hover, .next:hover {
  background-color: rgba(0,0,0,0.8);
  color: white;
}

/* The dot/bullet/indicator container */
.dot-container {
    text-align: center;
    padding: 20px;
    background: #ddd;
}

/* The dots/bullets/indicators */
.dot {
  cursor: pointer;
  height: 15px;
  width: 15px;
  margin: 0 2px;
  background-color: #bbb;
  border-radius: 50%;
  display: inline-block;
  transition: background-color 0.6s ease;
}

/* Add a background color to the active dot/circle */
.active, .dot:hover {
  background-color: #717171;
}

/* Add an italic font style to all quotes */
q {font-style: italic;}

/* Add a blue color to the author */
.author {color: cornflowerblue;}
</style>
</head>
<body>

<h1 align="center">Top 10 Recommended Products </h1></br>

<div class="slideshow-container">

"""+ recommend + """

<a class="prev" onclick="plusSlides(-1)">❮</a>
<a class="next" onclick="plusSlides(1)">❯</a>

</div>

<div class="dot-container">
"""+ dots + """
</div>

<script>
var slideIndex = 1;
showSlides(slideIndex);

function plusSlides(n) {
  showSlides(slideIndex += n);
}

function currentSlide(n) {
  showSlides(slideIndex = n);
}

function showSlides(n) {
  var i;
  var slides = document.getElementsByClassName("mySlides");
  var dots = document.getElementsByClassName("dot");
  if (n > slides.length) {slideIndex = 1}    
  if (n < 1) {slideIndex = slides.length}
  for (i = 0; i < slides.length; i++) {
      slides[i].style.display = "none";  
  }
  for (i = 0; i < dots.length; i++) {
      dots[i].className = dots[i].className.replace(" active", "");
  }
  slides[slideIndex-1].style.display = "block";  
  dots[slideIndex-1].className += " active";
}

</script>

</body>
</html> 
 
"""
# Display the HTML code in a Jupyter cell
display(HTML(html))